In [0]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.1.0


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
PROJECT_DIR = "/content/drive/My Drive/Colab Notebooks/cleansed_data.csv"
DATA_FILE_NAME = 'My Drive/Colab Notebooks/input.xlsx'
CLEANSED_FILE_DIR = '/My Drive/Colab Notebooks/'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


import warnings
warnings.filterwarnings('ignore')

In [0]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding , Activation, Bidirectional, Flatten, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False)
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred)

In [9]:
cleansed_data_df = load_doc("/content/drive/My Drive/Colab Notebooks/cleansed_data.csv")
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (5419, 5)


In [10]:
cleansed_data_df.describe()

,short_description,description,combined_desc,caller,assignment_group
count,5419,5419,5419,5419,5419
unique,3976,5419,5419,2351,52
top,,kabel,half gone applied half gone applied,rbozivdq gmlhrtvp,GRP_0
freq,306,1,1,46,2619


In [11]:
cleansed_data_df.head()

,short_description,description,combined_desc,caller,assignment_group
0,login issue,user manager checked user ad reset password ad...,login issue user manager checked user ad reset...,spxjnwir pjlcoqds,GRP_0
1,outlook,received team not outlook calendar somebody ad...,outlook received team not outlook calendar som...,hmjdrvpb komuaywn,GRP_0
2,cant log,received cannot log,cant log received cannot log,eylqgodm ybqkwiam,GRP_0
3,unable access page,unable access page,unable access page unable access page,xbkucsvz gcpydteq,GRP_0
4,error,error,error error,owlgqjme qhcozdfx,GRP_0


In [0]:
labelencoder = LabelEncoder()
cleansed_data_df['assignment_group'] = labelencoder.fit_transform(cleansed_data_df['assignment_group'])

In [13]:
cleansed_data_df

,short_description,description,combined_desc,caller,assignment_group
0,login issue,user manager checked user ad reset password ad...,login issue user manager checked user ad reset...,spxjnwir pjlcoqds,0
1,outlook,received team not outlook calendar somebody ad...,outlook received team not outlook calendar som...,hmjdrvpb komuaywn,0
2,cant log,received cannot log,cant log received cannot log,eylqgodm ybqkwiam,0
3,unable access page,unable access page,unable access page unable access page,xbkucsvz gcpydteq,0
4,error,error,error error,owlgqjme qhcozdfx,0
...,...,...,...,...,...
5414,option not,option not,option not option not,tmopbken ibzougsd,0
5415,fi two added,sorry another two need added copy problem erro...,fi two added sorry another two need added copy...,ipwjorsc uboapexr,2
5416,tablet needs due multiple,tablet needs due multiple,tablet needs due multiple tablet needs due mul...,cpmaidhj elbaqmtp,22
5417,not coming mail,received good afternoon not sent mail advise,not coming mail received good afternoon not se...,avglmrts vhqmtiua,21


In [0]:
cleansed_data = cleansed_data_df

In [15]:
oneHotencoder = OneHotEncoder()

#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
X = oneHotencoder.fit_transform(cleansed_data['assignment_group'].values.reshape(-1,1)).toarray()
print(X)
#print(cleansed_data.shape[1])
#To add this back into the original dataframe 
#dfOneHot = pd.DataFrame(X, columns = ["assignment_group_"+str(int(i)) for i in range(cleansed_data.shape[1])]) 
#df = pd.concat([cleansed_data, dfOneHot], axis=1)
#droping the country column 
#df= df.drop(['assignment_group'], axis=1) 
#printing to verify 
#print(df.head())


[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
from keras.layers.embeddings import Embedding

from sklearn.model_selection import train_test_split

# Split the data
x_train_keras, x_valid_keras, y_train_keras, y_valid_keras = train_test_split(cleansed_data_df['combined_desc'], X, test_size=0.2, shuffle= True)


In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_keras)
y_test_cate = to_categorical(y_valid_keras)
print(y_train_cate.shape)

x_train_keras = x_train_keras.astype(str)
x_valid_keras = x_valid_keras.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(list(x_train_keras))

#converting text into integer sequences
x_tr_seq_keras  = tokenizer.texts_to_sequences(x_train_keras) 
x_val_seq_keras = tokenizer.texts_to_sequences(x_valid_keras)

#padding to prepare sequences of same length
x_tr_seq_keras  = pad_sequences(x_tr_seq_keras, maxlen=100,padding='post')
x_val_seq_keras = pad_sequences(x_val_seq_keras, maxlen=100,padding='post')

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

(4335, 52, 2)
2872


In [18]:
print(x_train_keras[24])
print(x_tr_seq_keras[24])
print(x_train_keras[52])
print(x_tr_seq_keras[52])
print(x_tr_seq_keras[24])
print(x_tr_seq_keras[25])

password reset password reset
[148   5 858   5 126 405  11   9 248 109   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
outlook received
[159 227   6 159 227   6   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
[148   5 858   5 126 405  11   9 248 109   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  

In [41]:
#from keras.layers.embeddings import Embedding
#from keras.optimizers import Adam


model=Sequential()
#embedding layer
model.add(Embedding(size_of_vocabulary,100,input_length=100,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(Flatten())

#Dense Layer
#model.add(Dense(256,activation='relu'))  
model.add(Dense(128,activation='relu'))  
model.add(Dense(52,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-15, decay=0.0), loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  

mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 100)          287200    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 100, 256)          234496    
_________________________________________________________________
flatten_5 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               3276928   
_________________________________________________________________
dense_11 (Dense)             (None, 52)                6708      
Total params: 3,805,332
Trainable params: 3,805,332
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
model.fit(x_tr_seq_keras,y_train_keras,batch_size=32,epochs=100,validation_data=(x_val_seq_keras,y_valid_keras),verbose=1,callbacks=[es,mc])
#model.fit(x_tr_seq_fast,y_train_fast,batch_size=32,epochs=10,validation_data=(x_val_seq_fast,y_valid_fast),verbose=1,callbacks=[es,mc])

Epoch 1/100
136/136 [==============================] - ETA: 0s - loss: 2.4157 - acc: 0.4960
Epoch 00001: val_acc improved from -inf to 0.48339, saving model to best_model.h5
136/136 [==============================] - 36s 266ms/step - loss: 2.4157 - acc: 0.4960 - val_loss: 2.3297 - val_acc: 0.4834
Epoch 2/100
136/136 [==============================] - ETA: 0s - loss: 1.9713 - acc: 0.5412
Epoch 00002: val_acc improved from 0.48339 to 0.50830, saving model to best_model.h5
136/136 [==============================] - 40s 294ms/step - loss: 1.9713 - acc: 0.5412 - val_loss: 2.1687 - val_acc: 0.5083
Epoch 3/100
136/136 [==============================] - ETA: 0s - loss: 1.5999 - acc: 0.5991
Epoch 00003: val_acc improved from 0.50830 to 0.52214, saving model to best_model.h5
136/136 [==============================] - 36s 262ms/step - loss: 1.5999 - acc: 0.5991 - val_loss: 2.0375 - val_acc: 0.5221
Epoch 4/100
136/136 [==============================] - ETA: 0s - loss: 1.2820 - acc: 0.6713
Epoch 00

In [44]:
#loading best model
from tensorflow.keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_keras,y_valid_keras, batch_size=100)
print(val_acc)

11/11 [==============================] - 2s 158ms/step - loss: 2.0375 - acc: 0.5221
0.5221402049064636


In [50]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_keras)
y_test_cate = to_categorical(y_valid_keras)
print(y_train_cate.shape)

x_train_keras = x_train_keras.astype(str)
x_valid_keras = x_valid_keras.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(list(x_train_keras))

#converting text into integer sequences
x_tr_seq_keras  = tokenizer.texts_to_sequences(x_train_keras) 
x_val_seq_keras = tokenizer.texts_to_sequences(x_valid_keras)

#padding to prepare sequences of same length
x_tr_seq_keras_pre  = pad_sequences(x_tr_seq_keras, maxlen=100)
x_val_seq_keras_pre = pad_sequences(x_val_seq_keras, maxlen=100)

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

(4335, 52, 2)
2872


In [51]:
print(x_train_keras[24])
print(x_tr_seq_keras_pre[24])
print(x_train_keras[52])
print(x_tr_seq_keras_pre[52])
print(x_tr_seq_keras_pre[24])
print(x_tr_seq_keras_pre[25])

password reset password reset
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 148   5 858   5 126 405  11   9 248 109]
outlook received
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 159 227   6 159 227   6]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  

In [52]:
#from keras.layers.embeddings import Embedding
#from keras.optimizers import Adam


model=Sequential()
#embedding layer
model.add(Embedding(size_of_vocabulary,100,input_length=100,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(Flatten())

#Dense Layer
#model.add(Dense(256,activation='relu'))  
model.add(Dense(128,activation='relu'))  
model.add(Dense(52,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-15, decay=0.0), loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=15, verbose=1, mode='auto', baseline=None, restore_best_weights=False)  

mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 100)          287200    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100, 256)          234496    
_________________________________________________________________
flatten_6 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               3276928   
_________________________________________________________________
dense_13 (Dense)             (None, 52)                6708      
Total params: 3,805,332
Trainable params: 3,805,332
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
model.fit(x_tr_seq_keras_pre,y_train_keras,batch_size=32,epochs=10,validation_data=(x_val_seq_keras_pre,y_valid_keras),verbose=1,callbacks=[es,mc])

Epoch 1/10
136/136 [==============================] - ETA: 0s - loss: 0.7309 - acc: 0.8030
Epoch 00001: val_acc improved from 0.52583 to 0.53321, saving model to best_model.h5
136/136 [==============================] - 36s 264ms/step - loss: 0.7309 - acc: 0.8030 - val_loss: 2.6639 - val_acc: 0.5332
Epoch 2/10
136/136 [==============================] - ETA: 0s - loss: 0.5690 - acc: 0.8464
Epoch 00002: val_acc did not improve from 0.53321
136/136 [==============================] - 36s 264ms/step - loss: 0.5690 - acc: 0.8464 - val_loss: 2.7841 - val_acc: 0.5212
Epoch 3/10
136/136 [==============================] - ETA: 0s - loss: 0.4614 - acc: 0.8764
Epoch 00003: val_acc did not improve from 0.53321
136/136 [==============================] - 36s 263ms/step - loss: 0.4614 - acc: 0.8764 - val_loss: 3.0761 - val_acc: 0.5129
Epoch 4/10
136/136 [==============================] - ETA: 0s - loss: 0.3692 - acc: 0.9020
Epoch 00004: val_acc did not improve from 0.53321
136/136 [====================

In [57]:
#loading best model
from tensorflow.keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_keras_pre,y_valid_keras, batch_size=100)
print(val_acc)

11/11 [==============================] - 2s 157ms/step - loss: 2.6639 - acc: 0.5332
0.5332103371620178
